# Helmholtz Decomposition
Decompose a 2D wind field into rotational (ψ) and divergent (χ) components using pvtend's Helmholtz solver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvtend.helmholtz import helmholtz_decomposition
from pvtend.constants import R_EARTH

## 1. Create Synthetic Wind Field
Combine a vortex (rotational) and a source (divergent) into a single wind field.

In [ ]:
# Grid: 30°N–70°N, -40°E–40°E, 1° spacing
lat = np.arange(30, 71, 1.0)
lon = np.arange(-40, 41, 1.0)
lon2d, lat2d = np.meshgrid(lon, lat)
nlat, nlon = lat2d.shape

# Vortex centred at (50°N, 0°E)
dx_v = (lon2d - 0) * np.cos(np.deg2rad(50))
dy_v = lat2d - 50
r2 = dx_v**2 + dy_v**2 + 1
u_rot = -dy_v / r2 * 20
v_rot = dx_v / r2 * 20

# Divergent source at (45°N, -10°E)
dx_s = (lon2d + 10) * np.cos(np.deg2rad(45))
dy_s = lat2d - 45
r2_s = dx_s**2 + dy_s**2 + 4
u_div = dx_s / r2_s * 10
v_div = dy_s / r2_s * 10

u = u_rot + u_div
v = v_rot + v_div
print(f"Wind field shape: {u.shape}")

## 2. Helmholtz Decomposition
Use the DCT-based Poisson solver (fastest for limited-area domains).

In [ ]:
result = helmholtz_decomposition(u, v, lat, lon, method="dct")
print("Keys:", list(result.keys()))

## 3. Visualise Components

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
skip = 3  # quiver subsampling

titles = ["Original", "Rotational", "Divergent"]
u_list = [u, result["u_rot"], result["u_div"]]
v_list = [v, result["v_rot"], result["v_div"]]
bg_list = [None, result["psi"], result["chi"]]
bg_labels = ["", "ψ (streamfunction)", "χ (velocity potential)"]

for ax, title, uu, vv, bg, bgl in zip(axes, titles, u_list, v_list,
                                        bg_list, bg_labels):
    if bg is not None:
        im = ax.contourf(lon, lat, bg, levels=20, cmap="coolwarm", alpha=0.6)
        plt.colorbar(im, ax=ax, label=bgl, shrink=0.8)
    ax.quiver(lon[::skip], lat[::skip], uu[::skip, ::skip], vv[::skip, ::skip],
              scale=80, alpha=0.7)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

plt.tight_layout()
plt.show()

The rotational component captures the vortex flow (with streamfunction ψ), while the divergent component captures the source/sink (with velocity potential χ).